# LLM Personal NoteBook LangChain Framework  
## Basic API call for Mistral API

Here we will set our llm

In [36]:
from langchain_mistralai import ChatMistralAI
import os

llm = ChatMistralAI(
    api_key=os.environ['API_KEY'], 
    model="mistral-large-latest",
    temperature=0.1
)
response = llm.invoke("Hello, how are you?")
print(response.content)



Hello! I'm functioning as intended, thank you. How about you? How's your day going?


## Run Mistral Chat Test with structured output (e.g JSON)
Now We will use schema to render a json output of our model response </br>
To make sure that the model will render json format according to our need, </br>
We will use **pydentic**

In [34]:
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel

class AnswerWithOriginalMessageThenTraduction(BaseModel):
    original: str
    traduction: str

llm_structured_output = llm.with_structured_output(
    AnswerWithOriginalMessageThenTraduction,
    method='json_mode',
    include_raw=True
)
messages = [
    SystemMessage(
                  "Translate the following from English into Italian. "
                  "Make sure to return JSON blob with keys 'original' stands for "
                  "original text from prompt and 'traduction', stand for the traduction"
                  ),
    HumanMessage("hi?"),
]

response=llm_structured_output.invoke(messages)['raw']
print(response.content)



{
  "original": "hi?",
  "traduction": "ciao?"
}


### Result
```json
{
  "original": "hi!",
  "traduction": "ciao!"
}
```


## Prompt Template
Prompt Template is a technique that enable</br>
Developper to pass variable through prompt.</br>
Which means that promp is personnizable

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

# System template -> We tell the model what he will do here translate with any language
system_template = "Translate the following phrases from english to {language}"
output_format_template = """
Make sur to return JSON blob with keys 'original'
and keys 'traduction'
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", output_format_template),
        ("system", system_template),
        ("user", "{text}")
    ]
)

prompt = prompt_template.invoke({
    "language": "French",
    "text": "hi!"
})
message_template = prompt.to_messages()
response =llm_structured_output.invoke(message_template)
print(response['raw'].content)



{
  "original": "hi!",
  "traduction": "salut!"
}
